In [25]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("ExtractArtistNames") \
    .getOrCreate()

24/05/23 00:54:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [42]:
df = spark.read.csv("Staging/TopGlobalArtists50", header=True)
df.show()

+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       ArtistName|               album|            album_id|           artist_id|               track|            track_id|
+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             Feid|      FERXXOCALIPSIS|0lgs2Sa82lyX89nBU...|2LRoIwlKmHjgvigdN...|                LUNA|7bywjHOc0wSjGGbj0...|
|        ATL Jacob|      FERXXOCALIPSIS|0lgs2Sa82lyX89nBU...|7jAs1uPnpfNmT6e5q...|                LUNA|7bywjHOc0wSjGGbj0...|
|   Arctic Monkeys|                  AM|78bpIziExqiI9qztv...|7Ln80lUS6He07XvHI...|    I Wanna Be Yours|5XeFesFbtLpXzIVDN...|
|      Myles Smith|          Stargazing|3VNxRarx9HUI5w7DH...|3bO19AOone0ubCsfD...|          Stargazing|3Vr3zh0r7ALn8VLqC...|
|            ILLIT|       SUPER REAL ME|6irebIc6UO8fN0jl4...|36cgvBn0aadzOijnj...|            Magnetic|1aKvZDoLGkNMxoRYg...|


In [43]:
df = df.select("ArtistName", "artist_id")

In [44]:
df.show()

+-----------------+--------------------+
|       ArtistName|           artist_id|
+-----------------+--------------------+
|             Feid|2LRoIwlKmHjgvigdN...|
|        ATL Jacob|7jAs1uPnpfNmT6e5q...|
|   Arctic Monkeys|7Ln80lUS6He07XvHI...|
|      Myles Smith|3bO19AOone0ubCsfD...|
|            ILLIT|36cgvBn0aadzOijnj...|
|     Benson Boone|22wbnEMDvgVIAGdFe...|
|       Noah Kahan|2RQXRUsr4IW1f3mKy...|
|           Mitski|2uYWxilOVlUdk4oV9...|
|The Neighbourhood|77SW9BnxLY8rJ0Rci...|
|     Taylor Swift|06HL4z0CvFAxyc27G...|
|       Lord Huron|6ltzsmQQbmdoHHbLZ...|
| Luis R Conriquez|0pePYDrJGk8gqMRbX...|
|       Neton Vega|6pV5zH2LzjOUHaAvE...|
|     Taylor Swift|06HL4z0CvFAxyc27G...|
|       The Weeknd|1Xyo4u8uXC1ZmMpat...|
|           JENNIE|250b0Wlc5Vk0CoUsa...|
|   Lily-Rose Depp|1pBLC0qVRTB5zVMut...|
|           Future|1RyvyyTE3xzB2Zywi...|
|     Metro Boomin|0iEtIxbK0KxaSlF7G...|
|   Kendrick Lamar|2YZyLoL8N0Wb9xBt1...|
+-----------------+--------------------+
only showing top

In [45]:
df.write.mode('overwrite').options(header=True).saveAsTable('ArtistList')

24/05/23 01:35:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [46]:
import pandas as pd
import requests

# Read the CSV file containing artist names into a Pandas DataFrame
artist_names_df = pd.read_csv("spark-warehouse/artistlist")

# Function to retrieve artist information from Last.fm API
def get_artist_info(artist_name, api_key):
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getInfo&artist={artist_name}&api_key={api_key}&format=json"
    response = requests.get(url)
    artist_info = response.json().get("artist", {})
    artist_id = artist_info.get("mbid")
    overall_reach = artist_info.get("stats", {}).get("listeners")
    popularity = artist_info.get("stats", {}).get("playcount")
    genres = [tag["name"] for tag in artist_info.get("tags", {}).get("tag", [])]
    
    return {
        "ArtistName": artist_name,
        "ArtistID": artist_id,
        "OverallReach": overall_reach,
        "Popularity": popularity,
        "Genres": ", ".join(genres)
    }

# API key for Last.fm (replace with your own API key)
lastfm_api_key = "130ad717055f46130eObe133f8497c20"

# Initialize an empty list to store artist information
artist_info_list = []

# Iterate over the artist names in the DataFrame
for artist_name in artist_names_df["ArtistName"]:
    artist_info = get_artist_info(artist_name, lastfm_api_key)
    artist_info_list.append(artist_info)

# Convert the list of dictionaries to a Pandas DataFrame
artist_info_df = pd.DataFrame(artist_info_list)

# Display the DataFrame
print(artist_info_df)

# Save the DataFrame to a CSV file
# artist_info_df.to_csv("/path/to/save/artist_info_lastfm.csv", index=False)


IsADirectoryError: [Errno 21] Is a directory: 'spark-warehouse/artistlist'